# Detector de idiomas

Se va a diseñar un detector de idiomas utilizando recursos de **Machine Learning**. 
La idea principal es centrarlo en un problema de clasificación, con el que vamos a ayudarnos de una potente herramienta de ML para Python, **Scikit-Learn**.

In [1]:
import warnings
warnings.filterwarnings('ignore')
import numpy as np
import pandas as pd
from time import time
from sklearn import feature_extraction
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.externals import joblib
from xgboost import XGBClassifier
import lightgbm as lgb
import pickle

Vamos a trabajar con corpus del Parlamento Europeo que se encuentran disponibles en el siguiente enlace:
http://www.statmt.org/europarl/

Abrimos los diferentes corpus.
Aclaramos los idiomas:

it = italiano  
es = español  
en = inglés  
pt = portugués  
fr = francés

In [2]:
it = open('corpus/italiano','r', encoding='utf8')
es = open('corpus/espanol','r', encoding='utf8')
en = open('corpus/ingles','r', encoding='utf8')
fr = open('corpus/frances','r', encoding='utf8')
pt = open('corpus/portugues','r', encoding='utf8')

In [3]:
corpus_list = ['it', 'es', 'en', 'fr', 'pt']

Con la variable *size* le vamos a decir al programa la cantidad de frases con las que vamos a trabajar. Para que sea más fácil y todos los idiomas estén igualmente representados, procuraremos que *size* sea múltimo de 5

In [4]:
size = 10000*5

A continuación vamos a crear el DataFrame que estará constituido simplemente con dos columnas, la primera será una frase y la segunda el idioma de dicha frase.

In [5]:
t0 = time()

leng = []
txt = []

for j in corpus_list:
    n = 0
    for i in eval(j):
        if n < (int(size/len(corpus_list))):
            if len(i) >= 25:
                n+=1
                txt.append(i[:-1])
                leng.append(str(j))
        else:
             break
    eval(j).close()
                
df = pd.DataFrame()
df['texto'] = txt
df['idioma'] = leng

print("DataFrame creado en %0.4f s" % (time() - t0))

DataFrame creado en 0.0987 s


Podemos ver un pequeño resumen de nuestro DataFrame

In [6]:
df.describe()

,texto,idioma
count,50000,50000
unique,49693,5
top,Está encerrado o debate.,pt
freq,20,10000


In [7]:
df[df.idioma=='es'].head()

,texto,idioma
10000,Reanudación del período de sesiones,es
10001,Declaro reanudado el período de sesiones del P...,es
10002,"Como todos han podido comprobar, el gran ""efec...",es
10003,Sus Señorías han solicitado un debate sobre el...,es
10004,"A la espera de que se produzca, de acuerdo con...",es


In [8]:
df[df.idioma=='pt'].head()

,texto,idioma
40000,Declaro reaberta a sessão do Parlamento Europe...,pt
40001,"Como puderam constatar, o grande ""bug do ano 2...",pt
40002,Os senhores manifestaram o desejo de se proced...,pt
40003,"Entretanto, gostaria - como também me foi pedi...",pt
40004,Convido-os a levantarem-se para um minuto de s...,pt


Eliminamos las frases repetidas, ya que no son interesantes para este problema.

In [9]:
df.drop_duplicates(subset='texto', keep='first', inplace=True)

In [10]:
df.describe()

,texto,idioma
count,49693,49693
unique,49693,5
top,De sus trabajos surgirán la estructura y los m...,fr
freq,1,9948


Crearemos, a partir del DataFrame, nuestro conjunto de entrenamiento, train, y nuestro conjunto de validación, test.
Para realizar esto utilizaremos la herramienta de Scikit-Learn *train_test_split* cuyo parámetro *test_size* nos va a indicar el tamaño del conjunto de validación.

In [11]:
X_train, X_test, y_train, y_test = train_test_split(
    df['texto'],
    df['idioma'],
    test_size=0.4,
    random_state=0
)

In [12]:
X_test, X_val, y_test, y_val = train_test_split(
    X_test,
    y_test,
    test_size=0.5,
    random_state=0
)

In [13]:
X_train.shape, X_test.shape, X_val.shape

((29815,), (9939,), (9939,))

Con la funcion *feature_extraction.text.TfidfVectorizer* podremos separar la frase por palabras, lo que se conoce como *tokenizar*. El parámetro *analyzer='word'* nos indica que vamos a separar por palabras.

In [44]:
n_fts = 10000
vectorizer = feature_extraction.text.TfidfVectorizer(
    analyzer = 'word',
    ngram_range = (1, 1),
    max_features = n_fts,
    strip_accents = 'ascii',
    max_df = 0.8,
#     min_df = 0.01
)

Entrenamos un Pipeline con el TfIdf y un algoritmo RandomForest

In [56]:
clf = RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                             criterion='gini', max_depth=10, max_features='auto',
                             max_leaf_nodes=None, max_samples=0.8,
                             min_impurity_decrease=0.0, min_impurity_split=None,
                             min_samples_leaf=10, min_samples_split=5,
                             min_weight_fraction_leaf=0.0, n_estimators=200,
                             n_jobs=-1, oob_score=True, random_state=0, verbose=0,
                             warm_start=False)

In [57]:
model = Pipeline([
    ('vectorizer', vectorizer),
    ('model', clf)
])

print('Entrenando modelo {}'.format(clf))
model.fit(X_train, y_train)

Entrenando modelo RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=10, max_features='auto',
                       max_leaf_nodes=None, max_samples=0.8,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=10, min_samples_split=5,
                       min_weight_fraction_leaf=0.0, n_estimators=200,
                       n_jobs=-1, oob_score=True, random_state=0, verbose=0,
                       warm_start=False)


Pipeline(memory=None,
         steps=[('vectorizer',
                 TfidfVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.float64'>,
                                 encoding='utf-8', input='content',
                                 lowercase=True, max_df=0.8, max_features=10000,
                                 min_df=1, ngram_range=(1, 1), norm='l2',
                                 preprocessor=None, smooth_idf=True,
                                 stop_words=None, strip_accents='ascii',
                                 sublinear_tf=False,
                                 token_...
                 RandomForestClassifier(bootstrap=True, ccp_alpha=0.0,
                                        class_weight=None, criterion='gini',
                                        max_depth=10, max_features='auto',
                                        max_leaf_nodes=None, max_samples=0.8,
 

Grado de acierto en los diferentes conjuntos

In [58]:
print('Acierto en train:')
preds_tr = model.predict(X_train)
acc = accuracy_score(y_train, preds_tr)
print(acc,'\n')

print('Acierto en test:')
preds = model.predict(X_test)
acc = accuracy_score(y_test, preds)
print(acc,'\n')

print('Acierto en validación:')
preds_val = model.predict(X_val)
acc = accuracy_score(y_val, preds_val)
print(acc,'\n')

Acierto en train:
0.9844709039074292 

Acierto en test:
0.9847067109367139 

Acierto en validación:
0.9824932085722909 



In [59]:
cm = confusion_matrix(y_test, preds, labels=corpus_list)

Podemos visualizar una matriz de confusión para hacer una primera comprobación tanto de los errores como de los aciertos cometidos por el modelo

In [60]:
pd.DataFrame(cm,index=corpus_list,columns=corpus_list)

,it,es,en,fr,pt
it,1944,8,1,0,10
es,6,1893,1,2,63
en,1,0,1987,0,16
fr,7,17,0,1963,18
pt,0,1,1,0,2000


También podemos ver, a través de la función *classification_report* la precisión del modelo por cada idioma.

In [61]:
print(classification_report(y_test, preds,
                                    target_names=corpus_list))

              precision    recall  f1-score   support

          it       1.00      0.99      0.99      2004
          es       0.99      0.96      0.97      1965
          en       1.00      0.98      0.99      2005
          fr       0.99      0.99      0.99      1963
          pt       0.95      1.00      0.97      2002

    accuracy                           0.98      9939
   macro avg       0.99      0.98      0.98      9939
weighted avg       0.99      0.98      0.98      9939



Ahora podemos guardar nuestras predicciones en un dataframe.

In [62]:
predictions = pd.DataFrame()
predictions['texto'] = X_test
predictions['idioma'] = preds
predictions[0:10]

,texto,idioma
13026,"De conformidad con el orden del día, pasamos a...",es
1092,Vorrei chiedere alla Commissione se ha effettu...,it
8148,"Questo per me è assolutamente inaccettabile, p...",it
41615,"A Europol, a cooperação policial, não se encon...",pt
30344,Et pour arriver à un élan plus rapide et effic...,fr
29913,For despite these directives and frequent decl...,en
4414,Pensiamo che all'interno del concetto di multi...,it
12356,Pero en el contexto de dicha ayuda se debería ...,es
33790,"Cela parce que, premièrement, l' accent n' a p...",fr
16284,"De conformidad con el orden del día, se proced...",es


Guardamos el modelo entrenado para poder ejecutarlo en la aplicación Flask

In [63]:
joblib.dump(model, 'models/model.pkl', compress = 1)

['models/model.pkl']

In [64]:
new_model = joblib.load('models/model.pkl')

A continuación vamos a crear una función sencilla, lo que sería el detector en sí, donde el parámetro de entrada será una frase y la salida será el idioma de dicha frase.

In [65]:
corpus_list = ['it', 'es', 'en', 'fr', 'pt']

In [66]:
def detection(model, x):
    lenguage=model.predict([x])[0]
    proba = model.predict_proba([x])
    print(proba, max(proba[0]))
    if lenguage == 'fr':
        lenguage = 'francés'
    elif lenguage == 'pt':
        lenguage = 'portugués'
    elif lenguage == 'it':
        lenguage = 'italiano'
    elif lenguage == 'en':
        lenguage = 'inglés'
    elif lenguage == 'es':
        lenguage = 'español'
        
    print('El idioma de la frase es: '+lenguage)

Evaluemos algunos ejemplos:

In [67]:
detection(model, 'hola, mi nombre es Errodringer')

[[0.17061468 0.26047743 0.16121925 0.20523396 0.20245468]] 0.26047743392318173
El idioma de la frase es: español


In [68]:
detection(new_model, 'hola, mi nombre es Rodrigo')

[[0.17061468 0.26047743 0.16121925 0.20523396 0.20245468]] 0.26047743392318173
El idioma de la frase es: español


In [69]:
detection(new_model, 'suscríbete al canal para seguir aprendiendo')

[[0.16855631 0.21956045 0.16207203 0.216376   0.23343522]] 0.23343521575519052
El idioma de la frase es: portugués


In [70]:
detection(new_model, 'subscribe and like to continue learning')

[[0.31793718 0.16512693 0.15118934 0.17969129 0.18605527]] 0.3179371754801737
El idioma de la frase es: inglés


In [71]:
detection(new_model, 'hello, my name is Rodrigo')

[[0.26144984 0.18072633 0.16378003 0.19316904 0.20087478]] 0.26144983503826436
El idioma de la frase es: inglés


In [72]:
detection(new_model, 'Alguem aqui fala espanhol?')

[[0.18605718 0.19777842 0.18004677 0.21374507 0.22237256]] 0.2223725568718269
El idioma de la frase es: portugués


In [73]:
detection(new_model, 'Piacere di conoscerla')

[[0.16539579 0.17582645 0.16113572 0.29746113 0.20018091]] 0.2974611308082707
El idioma de la frase es: italiano
